In [ ]:
!pip install tensorflow-privacy

     |████████████████████████████████| 153kB 7.4MB/s 


In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2021-03-23 09:51:13--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-23 09:51:13--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [         <=>        ]  33.20M  6.10MB/s    in 16s     

2021-03-23 09:51:30 (2.04 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt
import torchvision
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class central_LeNet(nn.Module):
    def __init__(self, num_classes):
        super(central_LeNet, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=3, stride=1),
            nn.Tanh()
        )
        
        self.fc_classifier = nn.Sequential(
            nn.Linear(in_features=480, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes))
    
    def forward(self, t):
        if len(t.size()) != 4:
            raise Exception('Dimension of input incorrect!')
        t = self.feature_extractor(t)
        t = torch.flatten(t, 1)
        logits = self.fc_classifier(t)
        log_probs = F.log_softmax(logits)
        return logits, log_probs

In [ ]:
class central_Conv(nn.Module):
    def __init__(self, num_classes):
        super(central_Conv, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=3, stride=1),
            nn.Tanh()
        )
        
        self.fc_classifier = nn.Sequential(
            nn.Linear(in_features=480, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes))
    
    def forward(self, t):
        if len(t.size()) != 4:
            raise Exception('Dimension of input incorrect!')
        t = self.feature_extractor(t)
        t = torch.flatten(t, 1)
        logits = self.fc_classifier(t)
        log_probs = F.log_softmax(logits)
        return logits, log_probs

In [ ]:
class central_MLP(nn.Module):
    def __init__(self, num_classes):
        super(central_MLP, self).__init__()

        self.fc_classifier = nn.Sequential(
            nn.Linear(in_features=784, out_features=200),
            nn.ReLU(),
            nn.Linear(in_features=200, out_features=50),
            nn.ReLU(),
            nn.Linear(in_features=50, out_features=10)
        )

    def forward(self, t):
        t = t.view(t.shape[0], -1)
        t = self.fc_classifier(t)
        logits = t
        log_probs = F.log_softmax(t)
        return logits, log_probs

In [ ]:
# General class for each client (participant in the Federated Learning process)
class Client():
    def __init__(self, client_index, state_dict, DataLoader, architecture='MLP', batch_size=64, epochs=128, lr=0.01):
        """
        client_index: Identity for client k
        state_dict: Central model recieved from the server
        DataLoader: Loader for dataset
        architecture: String corresponding to which neural network architecture to use
        batch_size: Batch size for training
        epochs: Number of epochs to train
        lr: Learning rate for training
        """
        self.client_index = client_index
        # Load the central model for the client k
        self.model = model_dict[architecture](num_classes=num_classes)
        self.model.load_state_dict(state_dict)
        self.model.to(device)
        # Training hyperparameters
        self.num_epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.loader = DataLoader
        # Objective or Loss function
        self.objective = nn.NLLLoss()
        # Optimizer
        self.optimizer = optim.SGD(self.model.parameters(), lr = self.lr)

    # Function to update each client's local update
    def ClientUpdate(self, state_dict):
        # Store initial model before local update
        model_initial = state_dict
        self.model.load_state_dict(state_dict)
        self.model.to(device)

        # Multiple passes through the training data
        for epoch in range(self.num_epochs):
            # Switch to training mode
            self.model.train()
            epoch_loss = 0
            # For each batch
            for images, targets in self.loader:
                images, targets = images.to(device), targets.to(device)
                self.optimizer.zero_grad()
                # Forward propagation
                logits, log_probs = self.model.forward(images)
                loss = self.objective(log_probs, targets)
                # Backward propagation
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            # print('\nLoss in Training Epoch {} = {}\n'.format(epoch+1, epoch_loss))

        # client's local model update
        local_update_dict = {}
        # norm of update
        norm_dict = {}
        for key, value in model_initial.items():
            local_update_dict[key] = self.model.state_dict()[key] - value
            norm_dict[key] = torch.norm(input=local_update_dict[key], p=2)
        
        global dummy_global1
        global dummy_global2
        dummy_global1 = local_update_dict
        dummy_global2 = norm_dict
        return local_update_dict, norm_dict

In [ ]:
model_dict = {'MLP': central_MLP, 'CONV': central_Conv, 'LeNet': central_LeNet}
num_classes = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_clients = 100
deltas = {100: 0.001, 1000: 0.00001, 10000: 0.000001}
epsilon = 8
sigma = 1.12
batch_sizes = [32]*num_clients
epochs = [2]*num_clients
lrs = [0.01]*num_clients
clients_in_each_round = {100: 30, 1000: 300, 10000: 3000}
data_per_client = 600
central_models = []
statistics = []

In [ ]:
# model_dict, num_clients, 

In [ ]:
# General class for the centralized server or curator
class Server():
    def __init__(self, total_clients, privacy_budget, epsilon=epsilon, sigma=sigma, architecture='MLP'):
        """
        total_clients: Total number of clients in the federated learning process
        privacy_budget: Maximum privacy loss allowed for the group
        epsilon: Epsilon parameter in definition of Eps-Delta differential privacy
        sigma: Corresponds to sigma of the Gaussian mechanism. sigma = (Standard_Deviation_gaussian/Sensitivity_of_function)
        architecture: String corresponding to which neural network architecture to use
        """
        self.total_clients = total_clients
        self.privacy_budget = privacy_budget
        self.epsilon = epsilon
        self.sigma = sigma
        # Define central curator model according to the specified architecture
        self.central_model = model_dict[architecture](num_classes=num_classes).to(device)
        
        # Get each client's private training data
        self.samples = self.FetchSamples()
        
        # Instantiate the clients, load training and testing data
        self.clients = []
        for client_index in range(self.total_clients):
            trainloader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_sizes[client_index], sampler=self.samples[client_index])
            self.clients.append(Client(client_index, self.central_model.state_dict(), trainloader, architecture, batch_sizes[client_index], epochs[client_index], lrs[client_index]))
        self.testloader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_sizes[client_index])
        
        # Alphas in Renyi divergence (reference: https://arxiv.org/abs/1702.07476)
        self.orders = ([1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 3.0, 3.5, 4.0, 4.5] + list(range(5, 64)) + [128, 256, 512])

    # Get the current central model's performance on test data
    def GetModelPerformance(self):
        # Switch to evaluation mode
        self.central_model.eval()
        correctly_classified = 0
        for images, targets in self.testloader:
            images, targets = images.to(device), targets.to(device)
            logits, log_probs = self.central_model.forward(images)
            probs = torch.exp(log_probs)
            target_model_outputs = torch.argmax(probs, dim=1)
            # Number of data points correctly classified till now
            correctly_classified += torch.sum(target_model_outputs==targets).item()        
        return correctly_classified/len(mnist_test)

    # Fetches samples for each client's private training data
    def FetchSamples(self):
        samples = []
        indices = list(range(len(mnist_train)))
        np.random.shuffle(indices)
        # No overlap in the MNIST dataset
        if self.total_clients == 100:
            num_data_per_client = int(len(mnist_train)/self.total_clients)
            left_split = 0
            for client in range(self.total_clients):
                # Sample the data points using indices
                client_indices = indices[left_split:left_split+num_data_per_client]
                samples.append(SubsetRandomSampler(client_indices))
                left_split += num_data_per_client
        # Overlapping data points in MNIST dataset
        else:
            for client in range(self.total_clients):
                # Sample the data points using indices
                client_indices = np.random.choice(indices, data_per_client, replace=False)
                samples.append(SubsetRandomSampler(client_indices))
        return samples

    # Update the central model using Gaussian mechanism that approximates averaging while preserving privacy
    def CentralUpdate(self, client_updates, client_update_norms, state_dict):
        updated_state_dict = {}
        for key, value in state_dict.items():
            norms = []
            for client in range(len(client_update_norms)):
                norms.append(client_update_norms[client][key])
            # Senditivity == median of norms
            sensitivity = np.median(norms)
            updates_sum = 0
            for client in range(len(client_updates)):
                # Scaling using clipping
                clipping = max(1, float(client_update_norms[client][key]/sensitivity))
                updates_sum += (client_updates[client][key]/clipping)
            
            # Noise scaled to the sensitivity via Gaussian mechanism
            gaussian_mechanism = torch.normal(mean=0.0, std=float(sensitivity * self.sigma), size=updates_sum.size())        
            updates_sum += gaussian_mechanism
            # Updated parameters using Gaussian mechanism that approximate the sum of updates
            updated_state_dict[key] = value + (updates_sum/self.clients_in_each_round).float()
        return updated_state_dict
    
    # Central server (curator) execution
    def ServerExecution(self, clients_in_each_round):
        self.clients_in_each_round = clients_in_each_round
        # Number of interactions between server and clients
        communication_rounds = 0
        # List of central model performance after each communication round
        model_performances = []
        # List of privacy losses after each communication round
        privacy_losses = []
        # Total number of clients that participate in total number of communication rounds
        communication_cost = 0

        # Perform Federated learning until the privacy budget is reached
        while True:
            # counting the communication rounds between the clients and the server
            communication_rounds += 1
            # fraction of clients considered for this round
            sampling_ratio = float(self.clients_in_each_round/self.total_clients)
            # Get the Renyi differential privacy (reference: https://arxiv.org/abs/1702.07476)
            renyi_dp = compute_rdp(q=sampling_ratio, noise_multiplier=self.sigma, steps=communication_rounds, orders=self.orders)
            # Compute the privacy loss in the current communication round
            _, privacy_loss, _ = get_privacy_spent(orders=self.orders, rdp=renyi_dp, target_eps=epsilon)
            # Compute the performance (accuracy) of the central model
            model_performance = self.GetModelPerformance()

            print('Communication round: {}\n'.format(communication_rounds))
            model_performances.append(model_performance)
            privacy_losses.append(privacy_loss)
            print('Performance (accuracy) of the model: {}\n'.format(model_performance))
            print('Privacy loss (Delta spent): {}\n'.format(privacy_loss))
            print('Privacy budget (Delta upper bound): {}'.format(self.privacy_budget))
            print('\n###################################################################################\n')
            
            # Checks if the privacy loss till the current round has exceed the budget
            # If False: continue to the next communication round
            # If True: Abort the communication
            if privacy_loss > self.privacy_budget:
                print('\nPRIVACY BUDGET EXCEEDED!!\nABORTING THE FEDERATED LEARNING COMMUNICATION BETWEEN SERVER AND CLIENTS!!\n')
                break
            
            # Randomly pick "self.clients_in_each_round" number of client objects to participate in the federated learning process in this round
            participating_clients = np.random.choice(self.clients, self.clients_in_each_round)
            # Increment cost with the number of participating clients in this round
            communication_cost += len(participating_clients)
            
            # Stores local updates for all clients
            client_updates = []
            # Stores L2 norm of local updates for all clients
            client_update_norms = []
            for client in participating_clients:
                # Get local update and corresponding L2 norm of update for client
                client_update, client_update_norm = client.ClientUpdate(self.central_model.state_dict())
                client_updates.append(client_update)
                client_update_norms.append(client_update_norm)
            
            # Update the central model using Gaussian mechanism
            updated_central_model = self.CentralUpdate(client_updates, client_update_norms, self.central_model.state_dict())
            # Load the new central model parameters in the state_dict
            self.central_model.load_state_dict(updated_central_model)
        
        return self.central_model, (model_performances, communication_rounds, communication_cost, privacy_losses)

In [ ]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5,), (0.5,))])
mnist_train = torchvision.datasets.MNIST(root = './', train = True, download = True, transform = transform)
mnist_test = torchvision.datasets.MNIST(root = './', train = False, download = True, transform = transform)

In [ ]:
# Global server class
server = Server(total_clients=num_clients, privacy_budget=deltas[num_clients], epsilon=epsilon, sigma=sigma, architecture='MLP')
# Start executing the server with pre-specified number of clients in each round (selected randomly)
central_model, stats = server.ServerExecution(clients_in_each_round=clients_in_each_round[num_clients])
central_models.append(central_model)
statistics.append(stats)

Communication round: 1

Performance (accuracy) of the model: 0.1027

Privacy loss (Delta spent): 2.2895539214883055e-22

Privacy budget (Delta upper bound): 0.001

###################################################################################

Communication round: 2

Performance (accuracy) of the model: 0.1096

Privacy loss (Delta spent): 3.178553989532553e-14

Privacy budget (Delta upper bound): 0.001

###################################################################################

Communication round: 3

Performance (accuracy) of the model: 0.4213

Privacy loss (Delta spent): 1.5400643437611805e-11

Privacy budget (Delta upper bound): 0.001

###################################################################################

Communication round: 4

Performance (accuracy) of the model: 0.5029

Privacy loss (Delta spent): 3.718279649373213e-10

Privacy budget (Delta upper bound): 0.001

###################################################################################

Commun

In [ ]:
print(dummy_global1)

{'fc_classifier.0.weight': tensor([[ 2.1147e-05,  2.1140e-05,  2.1147e-05,  ...,  2.1130e-05,
          2.1126e-05,  2.1141e-05],
        [ 1.2035e-04,  1.2035e-04,  1.2035e-04,  ...,  1.2035e-04,
          1.2035e-04,  1.2035e-04],
        [ 3.5279e-05,  3.5286e-05,  3.5278e-05,  ...,  3.5284e-05,
          3.5279e-05,  3.5286e-05],
        ...,
        [ 3.7126e-05,  3.7126e-05,  3.7131e-05,  ...,  3.7133e-05,
          3.7126e-05,  3.7136e-05],
        [ 2.8543e-04,  2.8543e-04,  2.8543e-04,  ...,  2.8543e-04,
          2.8543e-04,  2.8543e-04],
        [-1.2188e-04, -1.2188e-04, -1.2188e-04,  ..., -1.2188e-04,
         -1.2188e-04, -1.2188e-04]]), 'fc_classifier.0.bias': tensor([-2.1147e-05, -1.2036e-04, -3.5284e-05, -1.8774e-05, -1.9393e-04,
         1.9714e-05, -1.0066e-04, -2.0097e-04, -1.3388e-04, -1.4100e-04,
        -1.9704e-05,  4.5338e-05,  3.0499e-04,  6.0765e-05, -5.8704e-05,
         8.9798e-06,  7.7382e-05,  1.6131e-04,  5.7691e-05, -9.8236e-06,
         0.0000e+00,  1.

In [ ]:
print(dummy_global2)

{'fc_classifier.0.weight': tensor(0.1078), 'fc_classifier.0.bias': tensor(0.0017), 'fc_classifier.2.weight': tensor(0.0647), 'fc_classifier.2.bias': tensor(0.0017), 'fc_classifier.4.weight': tensor(0.0528), 'fc_classifier.4.bias': tensor(0.0014)}


In [ ]:
def bin_list(a):
  res=[int(i) for i in bin(a)[2:]] 
  return res

In [ ]:
import math
a=dummy_global1['fc_classifier.0.weight'][0][0].item()
b=dummy_global1['fc_classifier.0.weight'][0][1].item()
print(a)
print(b)

2.1146610379219055e-05
2.11403239518404e-05


In [ ]:
global pt1
pt1=bin_list(math.ceil(a))
global pt2
pt2=bin_list(math.ceil(b))

Functions for random polynomial generation
```
binary_poly:-
Generates a polynomial with coeffecients in [0, 1]
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being
        the coeff of x ^ i.

 gen_uniform_poly:-
Generates a polynomial with coeffecients being integers in Z_modulus
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being
        the coeff of x ^ i.

gen_normal_poly:-
Generates a polynomial with coeffecients in a normal distribution
    of mean mean and a standard deviation std, then discretize it.
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being
        the coeff of x ^ i.


```

In [ ]:
def gen_binary_poly(size):
    return np.random.randint(0, 2, size, dtype=np.int64)


def gen_uniform_poly(size, modulus):
    return np.random.randint(0, modulus, size, dtype=np.int64)


def gen_normal_poly(size, mean, std):
    return np.int64(np.random.normal(mean, std, size=size))

Functions for polynomial evaluations 

```
polymul:-
Multiply two polynomials
    Args:
        x, y: two polynomials to be multiplied.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).

polyadd:-
Add two polynomials
    Args:
        x, y: two polynoms to be added.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
```



In [ ]:
def polymul(x, y, modulus, poly_mod):
    return np.int64(
        np.round(poly.polydiv(poly.polymul(x, y) %
                              modulus, poly_mod)[1] % modulus)
    )


def polyadd(x, y, modulus, poly_mod):
    
    return np.int64(
        np.round(poly.polydiv(poly.polyadd(x, y) %
                              modulus, poly_mod)[1] % modulus)
    )

Functions for polynomial evaluations continued

Description of Functions:-
```
polymul_wm:-
Multiply two polynomials
    Args:
        x, y: two polynomials to be multiplied.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z[X]/(poly_mod).

polyadd_wm:-
Add two polynomials
        Args:
            x, y: two polynomials to be added.
            poly_mod: polynomial modulus.
        Returns:
            A polynomial in Z[X]/(poly_mod).
```



In [ ]:
import numpy as np
from numpy.polynomial import polynomial as poly

def polymul_wm(x, y, poly_mod):
    return poly.polydiv(poly.polymul(x, y), poly_mod)[1]

def polyadd_wm(x, y, poly_mod):
    return poly.polydiv(poly.polyadd(x, y), poly_mod)[1]

Function for returning n's coefficients in base b ( lsb is on the left)



```
int2base:-
Generates the base decomposition of an integer n.
    Args:
        n: integer to be decomposed.
        b: base.
    Returns:
        array of coefficients from the base decomposition of n
        with the coeff[i] being the coeff of b ^ i.
 
```



In [ ]:
def int2base(n, b):
    if n < b:
        return [n]
    else:
        return [n % b] + int2base(n // b, b)   

Functions for keygen and relieanrization(converting quadratic polynomial to linear)

```
keygen:-
    Generate a public and secret keys
    Args:
        size: size of the polynoms for the public and secret keys.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
        std1: standard deviation of the error.
    Returns:
        Public and secret key.

evaluate_keygen_v1:-
Generate a relinearization key using version 1.
        Args:
            sk: secret key.
            size: size of the polynomials.
            modulus: coefficient modulus.
            T: base.
            poly_mod: polynomial modulus.
            std2: standard deviation for the error distribution.
        Returns:
            rlk0, rlk1: relinearization key.
```


In [ ]:
def keygen(size, modulus, poly_mod, std1):
    s = gen_binary_poly(size)
    a = gen_uniform_poly(size, modulus)
    e = gen_normal_poly(size, 0, std1)
    b = polyadd(polymul(-a, s, modulus, poly_mod), -e, modulus, poly_mod)
    return (b, a), s


def evaluate_keygen_v1(sk, size, modulus, T, poly_mod, std2):
    n = len(poly_mod) - 1
    l = np.int(np.log(modulus) / np.log(T))
    rlk0 = np.zeros((l + 1, n), dtype=np.int64)
    rlk1 = np.zeros((l + 1, n), dtype=np.int64)
    for i in range(l + 1):
        a = gen_uniform_poly(size, modulus)
        e = gen_normal_poly(size, 0, std2)
        secret_part = T ** i * poly.polymul(sk, sk)
        b = np.int64(polyadd(
        polymul_wm(-a, sk, poly_mod),
        polyadd_wm(-e, secret_part, poly_mod), modulus, poly_mod))

        b = np.int64(np.concatenate( (b, [0] * (n - len(b)) ) )) # pad b 
        a = np.int64(np.concatenate( (a, [0] * (n - len(a)) ) )) # pad a    

        rlk0[i] = b
        rlk1[i] = a
    return rlk0, rlk1


Function for encryption
```
Encrypt an integer.
    Args:
        pk: public-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        m: plaintext message, as an integer vector (of length <= size) with entries mod t.
    Returns:
        Tuple representing a ciphertext.
```


In [ ]:
def encrypt(pk, size, q, t, poly_mod, m, std1): 

    m = np.array(m + [0] * (size - len(m)), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m
    e1 = gen_normal_poly(size, 0, std1)
    e2 = gen_normal_poly(size, 0, std1)
    u = gen_binary_poly(size)
    ct0 = polyadd(
        polyadd(
            polymul(pk[0], u, q, poly_mod),
            e1, q, poly_mod),
        scaled_m, q, poly_mod
    )
    ct1 = polyadd(
        polymul(pk[1], u, q, poly_mod),
        e2, q, poly_mod
    )
    return (ct0, ct1)

Function for decryption

```
Decrypt a ciphertext.
    Args:
        sk: secret-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        ct: ciphertext.
    Returns:
        Integer vector representing the plaintext.
```

In [ ]:
def decrypt(sk, q, t, poly_mod, ct):
    scaled_pt = polyadd(
        polymul(ct[1], sk, q, poly_mod),
        ct[0], q, poly_mod
    )
    decrypted_poly = np.round(t * scaled_pt / q) % t
    return np.int64(decrypted_poly)

Function for adding and multiplying encrypted values(Encryption functions)

```
add_plain:-
Add a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer to add.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.

 add_cipher:-
Add a ciphertext and a ciphertext.
    Args:
        ct1, ct2: ciphertexts.
        q: ciphertext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.

mul_plain:-
Multiply a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer polynomial to multiply.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.
  
multiplication_coeffs:-
Multiply two ciphertexts.
        Args:
            ct1: first ciphertext.
            ct2: second ciphertext
            q: ciphertext modulus.
            t: plaintext modulus.
            poly_mod: polynomial modulus.
        Returns:
            Triplet (c0,c1,c2) encoding the multiplied ciphertexts.

mul_cipher_v1:-
Multiply two ciphertexts.
    Args:
        ct1: first ciphertext.
        ct2: second ciphertext
        q: ciphertext modulus.
        t: plaintext modulus.
        T: base
        poly_mod: polynomial modulus.
        rlk0, rlk1: output of the EvaluateKeygen_v1 function.
    Returns:
        Tuple representing a ciphertext.


```


In [ ]:
def add_plain(ct, pt, q, t, poly_mod):
    size = len(poly_mod) - 1
    m = np.array(pt + [0] * (size - len(pt)), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m
    new_ct0 = polyadd(ct[0], scaled_m, q, poly_mod)
    return (new_ct0, ct[1])


def add_cipher(ct1, ct2, q, poly_mod):
    new_ct0 = polyadd(ct1[0], ct2[0], q, poly_mod)
    new_ct1 = polyadd(ct1[1], ct2[1], q, poly_mod)
    return (new_ct0, new_ct1)


def mul_plain(ct, pt, q, t, poly_mod):
    size = len(poly_mod) - 1
    m = np.array(pt + [0] * (size - len(pt)), dtype=np.int64) % t
    new_c0 = polymul(ct[0], m, q, poly_mod)
    new_c1 = polymul(ct[1], m, q, poly_mod)
    return (new_c0, new_c1)

def multiplication_coeffs(ct1, ct2, q, t, poly_mod):
    c_0 = np.int64(np.round(polymul_wm(ct1[0], ct2[0], poly_mod) * t / q)) % q
    c_1 = np.int64(np.round(polyadd_wm(polymul_wm(ct1[0], ct2[1], poly_mod), polymul_wm(ct1[1], ct2[0], poly_mod), poly_mod) * t / q)) % q 
    c_2 = np.int64(np.round(polymul_wm(ct1[1], ct2[1], poly_mod) * t / q)) % q
    return c_0, c_1, c_2


def mul_cipher_v1(ct1, ct2, q, t, T, poly_mod , rlk0, rlk1):
    n = len(poly_mod) - 1
    l = np.int64(np.log(q) / np.log(T))  

    c_0, c_1, c_2 = multiplication_coeffs(ct1, ct2, q, t, poly_mod)
    c_2 = np.int64(np.concatenate( (c_2, [0] * (n - len(c_2))) )) #pad

    Reps = np.zeros((n, l + 1), dtype = np.int64)
    for i in range(n):
        rep = int2base(c_2[i], T)
        rep2 = rep + [0] * (l + 1 - len(rep)) #pad with 0
        Reps[i] = np.array(rep2, dtype=np.int64)
    c_20 = np.zeros(shape=n)
    c_21 = np.zeros(shape=n)
    for j in range(l + 1):
        c_20 = polyadd_wm(c_20, polymul_wm(rlk0[j], Reps[:,j], poly_mod), poly_mod)
        c_21 = polyadd_wm(c_21, polymul_wm(rlk1[j], Reps[:,j], poly_mod), poly_mod)

    c_20 = np.int64(np.round(c_20)) % q
    c_21 = np.int64(np.round(c_21)) % q

    new_c0 = np.int64(polyadd_wm(c_0, c_20, poly_mod)) % q
    new_c1 = np.int64(polyadd_wm(c_1, c_21, poly_mod)) % q

    return (new_c0, new_c1)


In [ ]:
    # polynomial modulus degree for RLWE(Ring learning with errors)
    n = 2 ** 2
    # ciphertext modulus 
    q = 2 ** 14
    # plaintext modulus
    t = 2
    # base for relin_v1
    T = int(np.sqrt(q)) 
    #modulusswitching modulus
    p = q ** 3

    # polynomial modulus
    poly_mod = np.array([1] + [0] * (n - 1) + [1])
    
    #standard deviation for the error in the encryption
    std1 = 1
    #standard deviation for the error in the evaluateKeyGen_v2
    std2 = 1

    # Keygen
    pk, sk = keygen(n, q, poly_mod, std1)

    #EvaluateKeygen_version1
    rlk0_v1, rlk1_v1 = evaluate_keygen_v1(sk, n, q, T, poly_mod, std1)
 
    # Encryption
    cst1, cst2 = [0, 1, 1, 0], [0, 1, 0, 0]

    ct1 = encrypt(pk, n, q, t, poly_mod, pt1, std1)
    ct2 = encrypt(pk, n, q, t, poly_mod, pt2, std1)

    print("[+] Ciphertext ct1 for plaintext pt1({}):".format(pt1))
    print("")
    print("\t ct1_0:", ct1[0])
    print("\t ct1_1:", ct1[1])
    print("")
    print("[+] Ciphertext ct2 for plaintext pt2({}):".format(pt2))
    print("")
    print("\t ct1_0:", ct2[0])
    print("\t ct1_1:", ct2[1])
    print("")
    
    cs1 = encrypt(pk, n, q, t, poly_mod, cst1, std1)
    cs2 = encrypt(pk, n, q, t, poly_mod, cst2, std1)
    print("[+] Ciphertext cs1 for plaintext cst1({}):".format(cst1))
    print("")
    print("\t cs1_0:", cs1[0])
    print("\t cs1_1:", cs1[1])
    print("")
    print("[+] Ciphertext cs2 for plaintext cst2({}):".format(cst2))
    print("")
    print("\t cs1_0:", cs2[0])
    print("\t cs1_1:", cs2[1])
    print("")

    # Evaluation
    ct3 = add_plain(ct1, cst1, q, t, poly_mod)
    ct4 = mul_plain(ct2, cst2, q, t, poly_mod)
    #ct5 = (ct1 + cst1) + (cst2 * ct2)
    ct5 = add_cipher(ct3, ct4, q, poly_mod)
    # ct6 = ct1 * ct2
    ct6 = mul_cipher_v1(ct1, ct2, q, t, T, poly_mod, rlk0_v1, rlk1_v1)
    # Decryption
    decrypted_ct3 = decrypt(sk, q, t, poly_mod, ct3)
    decrypted_ct4 = decrypt(sk, q, t, poly_mod, ct4)
    decrypted_ct5 = decrypt(sk, q, t, poly_mod, ct5)
    decrypted_ct6 = decrypt(sk, q, t, poly_mod, ct6)
  
    
    print("[+] Decrypted ct3=(ct1 + {}): {}".format(cst1, decrypted_ct3))
    print("[+] Decrypted ct4=(ct2 * {}): {}".format(cst2, decrypted_ct4))
    print("[+] Decrypted ct5=(ct1 + {} + {} * ct2): {}".format(cst1, cst2, decrypted_ct5))
    print("[+] pt1 + {} + {} * pt2): {}".format(cst1, cst2,polyadd(
                                                polyadd(pt1, cst1, t, poly_mod),
                                              polymul(cst2, pt2, t, poly_mod),
                                                t, poly_mod)))
    print("[+] Decrypted ct6=(ct1 * ct2): {}".format(decrypted_ct6))
    print("[+] pt1 * pt2: {}".format(polymul(pt1, pt2, t, poly_mod)))

[+] Ciphertext ct1 for plaintext pt1([1]):

	 ct1_0: [14646 14795 11350   702]
	 ct1_1: [ 7925 14380 12790  7757]

[+] Ciphertext ct2 for plaintext pt2([1]):

	 ct1_0: [13944  4866  9763 12053]
	 ct1_1: [  168  5919 10784  4164]

[+] Ciphertext cs1 for plaintext cst1([0, 1, 1, 0]):

	 cs1_0: [10785 12356  8025 10465]
	 cs1_1: [ 3763 14546  2325  2157]

[+] Ciphertext cs2 for plaintext cst2([0, 1, 0, 0]):

	 cs1_0: [15682 14647 14798 11351]
	 cs1_1: [ 8627  7924 14378 12790]

[+] Decrypted ct3=(ct1 + [0, 1, 1, 0]): [1 1 1]
[+] Decrypted ct4=(ct2 * [0, 1, 0, 0]): [0 1 0 0]
[+] Decrypted ct5=(ct1 + [0, 1, 1, 0] + [0, 1, 0, 0] * ct2): [1 0 1 0]
[+] pt1 + [0, 1, 1, 0] + [0, 1, 0, 0] * pt2): [1 0 1]
[+] Decrypted ct6=(ct1 * ct2): [1 0 0 0]
[+] pt1 * pt2: [1]
